In [ ]:
###############################
#PYTHON SCRIPT FOR LANDUSE CLASSIFICATION USING LANDSAT 8 LEVEL 1 IMAGES
#LANDSAT 8 LEVEL 1 DATA MUST BE ATMOSPHERICALLY CORRECTED AND PAN-SHARPENED USING SCP PLUGIN ON QGIS
#AUTHOR: PHAM DANG MANH HONG LUAN
#EMAIL: hongluanosgeo@gmail.com
#DATE: 09/04/2018
#VERSION: 1.0
###############################

In [ ]:
#IMPORT NECESSARY LIBRARIES
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="ticks", color_codes=True)
from osgeo import gdal,ogr,gdal_array
import pandas as pd
import sys,os,shutil
from sklearn.ensemble import RandomForestClassifier
import os.path
import geopandas as gpd
from sklearn.decomposition import PCA
from sklearn import preprocessing

In [ ]:
#GET DIMENSIONS OF INPUT DATA
indir = input('INPUT THE DIRECTORY TO SATELLITE DATA:')
while  os.path.isdir(indir) == False:
    print('\nDIRECTORY DOES NOT EXIST OR WAS INCORRECTLY INPUTED! PLEASE INPUT AGAIN!\n')
    indir = input('INPUT THE DIRECTORY TO SATELLITE DATA: ')
for dirname, dirnames, filenames in os.walk(indir):
    for filename in filenames:
        fullname = os.path.join(dirname, filename)
        if fullname[-3:].lower() == 'tif':
            if fullname[-5:].lower() == '2.tif':
                img = gdal.Open(fullname,gdal.GA_ReadOnly)
                gt = img.GetGeoTransform()
                ulX = gt[0]
                weres = gt[1]
                ulY = gt[3]
                nsres = gt[5]
                Xdim = img.RasterXSize
                Ydim = img.RasterYSize
                urX = ulX + Xdim * weres
                lrY = ulY - (Ydim * -(nsres))            

In [ ]:
#OPEN DATA
#INPUT DATA INCLUDES 6 BANDS OF LANDSAT IMAGE ATMOSPHERICALLY CORRECTED AND PAN-SHARPENED TO 15-M RESOLUTION
#Creat a list of input data
lstfullnames = []
for dirname, dirnames, filenames in os.walk(indir):
    for filename in filenames:
        fullname = os.path.join(dirname, filename)
        if fullname[-3:].lower() == 'tif':
            if fullname[-5:].lower() in ['2.tif','3.tif','4.tif','5.tif','6.tif','7.tif']:
                lstfullnames.append(fullname)
#Create array to store 6 bands
wedim = int((urX - ulX)/15)
nsdim = int((ulY - lrY)/15)
imgarray = np.zeros((nsdim, wedim, 6),dtype = np.float64)
#Load each band into newly created array
for j in range(len(lstfullnames)):          
    img = gdal.Open(lstfullnames[j],gdal.GA_ReadOnly)                     
    imgarray[:,:,j] = img.GetRasterBand(1).ReadAsArray()               

In [ ]:
#CLIP INPUT DATA
#Import shapefile of region of interest
subdir = input('INPUT THE NAME OF SHAPEFILE TO SUBSET DATA: ')
while  os.path.isfile(subdir) == False:
    print('\nFILE DOES NOT EXIST OR WAS INCORRECTLY INPUTED! PLEASE INPUT AGAIN!\n')
    subdir = input('INPUT THE NAME OF SHAPEFILE TO SUBSET DATA: ')
driver = ogr.GetDriverByName("ESRI Shapefile")
datasource = driver.Open(str(subdir),0)
lyr = datasource.GetLayer() 
#Get spatial extensions of input shapefile
XshpMin,XshpMax,YshpMin,YshpMax = lyr.GetExtent()
#Converting spatial extensions to array extensions
#Function getting array extension
def geo2arrdim(Xmin,Ymax,X,Y):
    ulX = Xmin
    weres = 15
    ulY = Ymax
    nsres = -15
    Xcord = int((X - ulX)/weres)
    Ycord = int((Y - ulY)/nsres)
    return(Xcord, Ycord)
#Convert shapefile extensions to array extensions
ulX2,ulY2 = geo2arrdim(ulX,ulY,XshpMin, YshpMax)
lrX2,lrY2 = geo2arrdim(ulX,ulY,XshpMax, YshpMin)
#Create array of clipped raster bands
cliparr = np.zeros((lrY2-ulY2,lrX2-ulX2 ,6),dtype = np.float64)
for i in range(imgarray.shape[2]):            
    cliparr[:,:,i] = imgarray[ulY2:lrY2,ulX2:lrX2,i]

In [ ]:
#VISUALIZING CLIPED INPUT LANDSAT BANDS
bandlist = ['Blue', 'Green', 'Red', 'Near Infrared', 'SWIR 1', 'SWIR 2']
fig= plt.figure(figsize=(25,10))
for i in range(6):       
    plt.subplot(2,3,i+1)
    im=plt.imshow(cliparr[:,:,i],cmap=plt.cm.Greys_r)
    plt.title('{} Band'.format(bandlist[i]))
    plt.colorbar(im)
    plt.axis('off')
plt.show()

In [ ]:
#VISUALIZING HISTOGRAM OF INPUT LANDSAT BANDS
#Convert to two-dimensional array
def convert2sing(imgarray):
    arrlist = []
    for i in range(imgarray.shape[2]):
        barray = imgarray[:,:,i]
        singarray = barray.ravel()
        arrlist.append(singarray)
    return arrlist
singarrlst = convert2sing(cliparr)
#Plot histogrms of input Landsat bands
colors = ['blue','green','red','cyan','magenta','yellow']
fig = plt.figure(figsize=(25,10))
plt.hist(singarrlst, bins = 200, color = colors, label = bandlist,histtype='step')
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102),fontsize = 15,loc = 2, ncol=6, mode='expand')
plt.xlabel('Pixel value', fontsize = 15)
plt.ylabel('Number of pixels', fontsize = 15)
plt.tick_params(axis='both', labelsize=15)
plt.show() 

In [ ]:
#CREATING COMPOSITE IMAGE
#REMEMBER THAT NEW ORDER OF LANDSAT 8 BANDS IN NEWLY CREATEDLY ARRAY IS AS FOLLOW (ORDER ARRAY/NAME OF BAND)
#0/BLUE (band 2); 1/GREEN (band 3); 2/RED (band 4); 3/NEAR INFRARED (band 5); 4/SWIR 1 (band 6); 5/SWIR 2 (band 7)
#Function normalizing bands
def normbands(imgarr):
    clipnorm = np.zeros((cliparr.shape[0],cliparr.shape[1],cliparr.shape[2]),dtype=np.float64)
    for a in range(cliparr.shape[2]):
        std = cliparr[:,:,a].std()
        mean = cliparr[:,:,a].mean()
        minval = mean - std
        maxval = mean + std
        for i in range(cliparr.shape[0]):
            for j in range(cliparr.shape[1]):
                if cliparr[i,j,a] > maxval:
                    clipnorm[i,j,a] = maxval
                elif cliparr[i,j,a] < minval:
                    clipnorm[i,j,a] = minval
                else:
                    clipnorm[i,j,a] = cliparr[i,j,a]
    for b in range(clipnorm.shape[2]):
        clipnorm[:,:,b] = (clipnorm[:,:,b]-clipnorm[:,:,b].min())*1/(clipnorm[:,:,b].max() - clipnorm[:,:,b].min())
    return clipnorm
#Get normalized array
clipnorm = normbands(cliparr)
#Create composite image
compnatimg = np.dstack((clipnorm[:,:,2],clipnorm[:,:,1],clipnorm[:,:,0]))
compinfimg = np.dstack((clipnorm[:,:,3],clipnorm[:,:,2],clipnorm[:,:,1]))
comphveimg = np.dstack((clipnorm[:,:,4],clipnorm[:,:,3],clipnorm[:,:,0]))
compveaimg = np.dstack((clipnorm[:,:,4],clipnorm[:,:,3],clipnorm[:,:,2]))
compagrimg = np.dstack((clipnorm[:,:,4],clipnorm[:,:,3],clipnorm[:,:,2]))
compufcimg = np.dstack((clipnorm[:,:,5],clipnorm[:,:,4],clipnorm[:,:,2]))
compswiimg = np.dstack((clipnorm[:,:,5],clipnorm[:,:,3],clipnorm[:,:,2]))
complwimg = np.dstack((clipnorm[:,:,3],clipnorm[:,:,4],clipnorm[:,:,2]))
#Visualizing natural composite image
figure = plt.figure(figsize=(25,10))
plt.imshow(compnatimg)
plt.title('Natural Composite', fontsize = 15)
plt.axis('off')
plt.tight_layout    
plt.show()

In [ ]:
#Visualizing infrared composite image
figure = plt.figure(figsize=(25,10))
plt.imshow(compinfimg)
plt.title('Color Infrared Composite', fontsize = 15)
plt.axis('off')
plt.tight_layout    
plt.show()

In [ ]:
#Visualizing infrared composite image
figure = plt.figure(figsize=(25,10))
plt.imshow(comphveimg)
plt.title('Healthy Vegitation Composite', fontsize = 15)
plt.axis('off')
plt.tight_layout    
plt.show()

In [ ]:
#Visualizing infrared composite image
figure = plt.figure(figsize=(25,10))
plt.imshow(compveaimg)
plt.title('Vegitation Analysis Composite', fontsize = 15)
plt.axis('off')
plt.tight_layout    
plt.show()

In [ ]:
#Visualizing infrared composite image
figure = plt.figure(figsize=(25,10))
plt.imshow(compagrimg)
plt.title('Agricultural Composite', fontsize = 15)
plt.axis('off')
plt.tight_layout    
plt.show()

In [ ]:
#Visualizing urban false color composite image
figure = plt.figure(figsize=(25,10))
plt.imshow(compufcimg)
plt.title('Urban False Color Composite', fontsize = 15)
plt.axis('off')
plt.tight_layout    
plt.show()

In [ ]:
#Visualizing urban false color composite image
figure = plt.figure(figsize=(25,10))
plt.imshow(compswiimg)
plt.title('Shortwave Infrared Composite', fontsize = 15)
plt.axis('off')
plt.tight_layout    
plt.show()

In [ ]:
#Visualizing urban false color composite image
figure = plt.figure(figsize=(25,10))
plt.imshow(complwimg)
plt.title('Land/Water Composite', fontsize = 15)
plt.axis('off')
plt.tight_layout    
plt.show()

In [ ]:
#CALCULATE NDVI AND NDWI
#REMEMBER THAT NEW ORDER OF LANDSAT 8 BANDS IN NEWLY CREATEDLY ARRAY IS AS FOLLOW (ORDER ARRAY/NAME OF BAND)
#0/BLUE; 1/GREEN; 2/RED; 3/NEAR INFRARED; 4/SWIR 1; 5/SWIR 2
#Calculate NDVI
ndvi = (cliparr[:,:,3]-cliparr[:,:,2])/(cliparr[:,:,3]+cliparr[:,:,2])
#Calculate NDWI
ndwi = (cliparr[:,:,1]-cliparr[:,:,3])/(cliparr[:,:,1]+cliparr[:,:,3])

In [ ]:
#VISUALIZING NDVI
figure = plt.figure(figsize=(25,10))
im = plt.imshow(ndvi,cmap=plt.cm.rainbow)
plt.title('NDVI', fontsize = 15)
plt.axis('off')
plt.colorbar(im)   
plt.show()

In [ ]:
#VISUALIZING NDVI
figure = plt.figure(figsize=(25,10))
im = plt.imshow(ndwi,cmap=plt.cm.rainbow)
plt.title('NDWI', fontsize = 15)
plt.axis('off')
plt.colorbar(im)
plt.tight_layout    
plt.show()

In [ ]:
#CREATE NEW ARRAY WITH NDVI AND NDWI DATA
cliparr2 = np.zeros((lrY2-ulY2,lrX2-ulX2 ,8),dtype = np.float64)
for i in range(cliparr.shape[2]):
    cliparr2[:,:,i] = cliparr[:,:,i]
cliparr2[:,:,6] = ndvi
cliparr2[:,:,7] = ndwi

In [ ]:
#VISUALIZING ALL INPUT DATA
bandlist = ['Blue', 'Green', 'Red', 'Near Infrared', 'SWIR 1', 'SWIR 2','NDVI','NDWI']
fig= plt.figure(figsize=(25,10))
for i in range(6):       
    plt.subplot(3,3,i+1)
    im=plt.imshow(cliparr2[:,:,i],cmap=plt.cm.Greys_r)
    plt.title('{} Band'.format(bandlist[i]))
    plt.colorbar(im)
    plt.axis('off')
#Visualizing NDVI   
plt.subplot(3,3,7)
im=plt.imshow(cliparr2[:,:,6],cmap=plt.cm.rainbow)
plt.title('{}'.format(bandlist[6]))
plt.colorbar(im)
plt.axis('off')
#Visualizing NDWI
plt.subplot(3,3,8)
im=plt.imshow(cliparr2[:,:,7],cmap=plt.cm.rainbow)
plt.title('{}'.format(bandlist[7]))
plt.colorbar(im)
plt.axis('off')
plt.show()

In [ ]:
#VISUALIZING PAIRWISE PLOT
#Get list of single-dimensional array
singlist2 = convert2sing(cliparr2)                
#Function creating dataframe of input array
def array2df(singlist2):
    df = pd.DataFrame()
    df['Blue'] = singlist2[0]
    df['Green'] = singlist2[1]
    df['Red'] = singlist2[2]
    df['Near Infrared'] = singlist2[3]
    df['SWIR 1'] = singlist2[4]
    df['SWIR 2'] = singlist2[5]
    df['NDVI'] = singlist2[6]
    df['NDWI'] = singlist2[7]
    return df
#Create new dataframe storing input array
arrdf = array2df(singlist2)
#Draw pairwise plot of input array
fig = plt.figure(figsize=(25,10))
sns.pairplot(arrdf)
plt.show()

In [ ]:
#CREATING TRAINING DATA
#Input training shapefile
traindir = input('INPUT DIRECTORY AND NAME OF SHAPEFILE OF TRAINING DATA: ')
while  os.path.isfile(traindir) == False:
    print('\nDIRECTORY DOES NOT EXIST OR WAS INCORRECTLY INPUTED! PLEASE INPUT AGAIN!\n')
    traindir = input('INPUT DIRECTORY AND NAME OF SHAPEFILE OF TRAINING DATA: ')
driver = ogr.GetDriverByName("ESRI Shapefile")    
datasource = driver.Open(str(traindir),0)
lyr = datasource.GetLayer()
#Function get list of fields of shapefile
def getfield(inshp):
    trainshp = gpd.read_file(inshp)
    return   trainshp.columns.values
#Choose ID field (Field that hold ID number of landuse class)
field = input('\nINPUT NAME OF ID FIELD: ')
while field not in getfield(traindir):
    print('\nFIELD DOES NOT EXIST OR WRONGLY INPUTED!\n')
    field = input('INPUT NAME OF ID FIELD: ')
#Rasterize training shapefile using ID field
#Create raster properties
#Create spatial properties
nrow = cliparr2.shape[0]
ncol = cliparr2.shape[1]
proj = img.GetProjectionRef()
weres = 15
nsres = 15
ulX = XshpMin
ulY = YshpMax
gt = (ulX, weres, 0, ulY, 0, -nsres)
#Create raster layer in memory
driver = gdal.GetDriverByName('MEM')
outrast = driver.Create('',ncol,nrow,gdal.GDT_Byte)
outrast.SetProjection(proj)
outrast.SetGeoTransform(gt)    
b = outrast.GetRasterBand(1)
gdal.RasterizeLayer(outrast,[1],lyr,options = ['ATTRIBUTE={}'.format(field)])
trainarr = outrast.GetRasterBand(1).ReadAsArray()
#Convert array of training data to single-dimensional array
trainsingarr = trainarr.ravel()

In [ ]:
#VISUALIZING PAIRWISE PLOT OF TRAINING DATA
#Creating dataframe of input array and training data
arrdf['Class'] = trainsingarr
traindf = arrdf.loc[arrdf['Class'] > 0]
#Draw pairwise plot of input array and training data
fig = plt.figure(figsize=(25,10))
sns.pairplot(traindf, hue = 'Class')
plt.show()

In [ ]:
#DRAW BOX-PLOT OF TRAINING DATA
for band in bandlist:  
    plt.figure
    dfb = traindf[[band, 'Class']]
    dfb.boxplot(column=band, by='Class', figsize=(12,10))
    plt.show()

In [ ]:
#CREATE INPUTS FOR CLASSIFICATION ALGORITHM
#Create array of input data
inarray = arrdf[['Blue', 'Green', 'Red', 'Near Infrared', 'SWIR 1', 'SWIR 2','NDVI','NDWI']].values
#Create inputs for classification algorithm
X = inarray[trainsingarr>0]
y = trainsingarr[trainsingarr>0]

In [ ]:
#CLASSIFICATION
rf = RandomForestClassifier(n_estimators = 500, oob_score=True, class_weight = 'balanced')
rf = rf.fit(X,y)
class_prediction = rf.predict(inarray)
class_prediction = class_prediction.reshape(cliparr[:, :, 0].shape)


In [ ]:
#VISUALIZING CLASSIFICATION RESULT
plt.figure(figsize = (25,10))
im = plt.imshow(class_prediction,cmap=plt.cm.Paired)
plt.title('RANDOM FOREST', fontsize = 15)
plt.colorbar(im)
plt.axis('off')
plt.show()

In [ ]:
#EXPORTING CLASSIFICATION RESULT TO TIFF FILE
outrast = input('\nPLEASE INPUT THE DIRECTORY AND NAME OF OUTPUT RASTER (WITH .TIF EXTENSION): ')
while  os.path.isdir(os.path.dirname(outrast)) == False:
    print('\nDIRECTORY DOES NOT EXIST OR WAS INCORRECTLY INPUTED! PLEASE INPUT AGAIN!\n')
    outrast = input('PLEASE INPUT THE DIRECTORY AND NAME OF OUTPUT RASTER: ')
cols = cliparr2.shape[1]
rows = cliparr2.shape[0]
driver2 = gdal.GetDriverByName('GTiff')
outRaster = driver2.Create(outrast, cols, rows, 1, gdal.GDT_Byte)
outRaster.SetGeoTransform((XshpMin, 15, 0, YshpMax, 0, -15))
outRaster.SetProjection(proj)
outRaster.GetRasterBand(1).WriteArray(class_prediction)
outRaster.FlushCache()